In [ ]:
#wd<-"/home/jovyan/zxli_SCI/result/RCTD/testC5/WT_all/fc1.fcreg1.conf5_220317/"
wd<-"/result/RCTD/testC5/WT_all/fc1/"
if(!dir.exists(wd))
    dir.create(wd)
setwd(wd)
getwd()

In [2]:
source("/script/SPOTlightHelper-Copy1.R")
source("/script/DotPlot_ct.R")

In [ ]:
count_interactions <- function(x, min_prop) {
    # for each pair of groups count how many
    # samples have value above 'min_prop'
    x <- x > min_prop
    ij <- combn(colnames(x), 2)
    y <- apply(ij, 2, function(.) sum(rowAlls(x[, ., drop = FALSE])))

    # construct 'data.frame'
    df <- data.frame(t(ij), y)
    names(df) <- c("from", "to", "n")
    return(df)
}
               
plot_network <- function(x, df, ...) {
    # Check necessary packages are installed and if not STOP
    #.test_installed("igraph")

    w <- scale(df$n, 1)
    g <- igraph::graph_from_data_frame(df,
        vertices = colnames(x),
        directed = FALSE)
    
    igraph::plot.igraph(g, edge.width = w, ...)
}

In [ ]:
library(spacexr)
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
#library(hdf5r)
library(png)
library(viridis)
library(ggsci)
library(ggalluvial)
#library(tidyverse)
#library(clustree)
library(RCTD)
library(Matrix)
#library(SPOTlight)
library(future)
library(grid)
library(RColorBrewer)
#library(pheatmap)
library(circlize)
library(igraph)
library(ComplexHeatmap)
library(stringr)
library(ComplexHeatmap)
library(cowplot)
library(scales)
library(tidyr)
library(scater)
library(scran)
library(reshape2)

In [20]:
get_marker_data <- function(cell_type_names, cell_type_means, gene_list) {
  marker_means = cell_type_means[gene_list,]
  marker_norm = marker_means / rowSums(marker_means)
  marker_data = as.data.frame(cell_type_names[max.col(marker_means)])
  marker_data$max_epr <- apply(cell_type_means[gene_list,],1,max)
  colnames(marker_data) = c("cell_type",'max_epr')
  rownames(marker_data) = gene_list
  marker_data$log_fc <- 0
  epsilon <- 1e-9
  for(cell_type in unique(marker_data$cell_type)) {
    cur_genes <- gene_list[marker_data$cell_type == cell_type]
    other_mean = rowMeans(cell_type_means[cur_genes,cell_type_names != cell_type])
    marker_data$log_fc[marker_data$cell_type == cell_type] <- log(epsilon + cell_type_means[cur_genes,cell_type]) - log(epsilon + other_mean)
  }
  return(marker_data)
}

In [9]:
reference<-readRDS("/result/RCTD/ref_10x_astr/L3_taxon/Astrocyte_integrate_split2A/2reference.3astr.labeltransfer.L3.astrocyte_RCTD.reference.seed1234.down1000.without7dpi.rds")

In [12]:
puck<-readRDS("/result/RCTD/WT.replace_v2.SCT.puck.rds")

In [ ]:
myRCTD <- create.RCTD(puck, reference,CELL_MIN_INSTANCE = 10,max_cores=10,
                      UMI_min = (min(puck@nUMI)-1),UMI_max = 200000000,fc_cutoff=1,fc_cutoff_reg = 1)
#myRCTD@config$CONFIDENCE_THRESHOLD<-5
myRCTD.multi <- run.RCTD(myRCTD, doublet_mode = 'multi')
saveRDS(myRCTD.multi,'myRCTD_visium_multi.rds')

### get the cell type highest marker list and df

In [ ]:
sc_exp<-myRCTD@cell_type_info$info[[1]]
#sc_exp$gene<-rownames(sc_exp)
head(sc_exp)
sc_df<-get_marker_data(myRCTD@cell_type_info$info[[2]],myRCTD@cell_type_info$info[[1]],myRCTD@internal_vars$gene_list_bulk)
sc_df$gene<-rownames(sc_df)
head(sc_df)
dim(sc_df)
range(sc_df$log_fc)
table(sc_df$cell_type)
sc_exp_norm<-sc_exp[sc_df$gene,]
dim(sc_exp_norm)
head(sc_exp_norm)
sc_exp_norm<-sc_exp_norm / rowSums(sc_exp_norm)
sc_exp_norm$gene<-rownames(sc_exp_norm)
head(sc_exp_norm)
table(sc_exp_norm$gene==sc_df$gene)
dev_marker_df<-cbind(sc_exp_norm,sc_df)
head(dev_marker_df)


In [22]:
write.csv(dev_marker_df,"fc1.fcreg1.deconvolution.celltype.marker.normalized.data.frame.csv")

### get the deconvolution dataframe

In [ ]:
result<-myRCTD.multi@results
length(result)

In [16]:
sub_weights <- data.frame(result[[1]]$sub_weights)
sub_weights$ct <- row.names(sub_weights)
colnames(sub_weights)[1]<-1

sub_weights_conf <- data.frame(result[[1]]$sub_weights[result[[1]]$conf_list])
sub_weights_conf$ct <- row.names(sub_weights_conf)
colnames(sub_weights_conf)[1]<-1
for(i in seq(2, length(result))){
    tmp <- data.frame(result[[i]]$sub_weights[result[[i]]$conf_list])
    tmp$ct <- row.names(data.frame(tmp))
    colnames(tmp)[1]<-i
    sub_weights_conf <- merge(x = sub_weights_conf, y =tmp, by = "ct", all = T)
    
    tmp2 <- data.frame(result[[i]]$sub_weights)
    tmp2$ct <- row.names(data.frame(tmp2))
    colnames(tmp2)[1]<-i
    sub_weights <- merge(x = sub_weights, y =tmp2, by = "ct", all = T)
}

In [ ]:
sub_weights[is.na(sub_weights)] <- 0
sub_weights[1:5,1:5]

sub_weights_conf[is.na(sub_weights_conf)] <- 0
sub_weights_conf[1:5,1:5]

In [ ]:
dec_mtx<-sub_weights
rownames(dec_mtx)<-dec_mtx$ct
dec_mtx<-dec_mtx[,-1]
dec_mtx.t<-t(dec_mtx)
head(dec_mtx.t)

In [ ]:
dec_mtx_conf<-sub_weights_conf
rownames(dec_mtx_conf)<-dec_mtx_conf$ct
dec_mtx_conf<-dec_mtx_conf[,-1]
dec_mtx_conf.t<-t(dec_mtx_conf)
head(dec_mtx_conf.t)

In [ ]:
### normalize confident dec_mtx
empty_cells<-rownames(dec_mtx_conf.t)[rowSums(dec_mtx_conf.t)==0]
length(empty_cells)
empty_cells

dec_mtx_conf.t_nor<-normalize_weights(dec_mtx_conf.t)
dec_mtx_conf.t_nor[empty_cells,]<-0
table(rowSums(dec_mtx_conf.t_nor)==0)

In [ ]:
rownames(dec_mtx.t)<-colnames(puck@counts)
head(dec_mtx.t)

rownames(dec_mtx_conf.t_nor)<-colnames(puck@counts)
head(dec_mtx_conf.t_nor)

In [ ]:
colnames(dec_mtx.t)<-gsub("[[:punct:]]",".",colnames(dec_mtx.t))
colnames(dec_mtx.t)<-gsub(" ",".",colnames(dec_mtx.t))
colnames(dec_mtx.t)

In [ ]:
colnames(dec_mtx_conf.t_nor)<-gsub("[[:punct:]]",".",colnames(dec_mtx_conf.t_nor))
colnames(dec_mtx_conf.t_nor)<-gsub(" ",".",colnames(dec_mtx_conf.t_nor))
colnames(dec_mtx_conf.t_nor)

In [30]:
write.csv(dec_mtx.t, "WT.fc1.fcreg1_subweights.decon.mtx.csv")
write.csv(dec_mtx_conf.t_nor, "WT.fc1.fcreg1_subweights_conf_normalized.decon.mtx.csv")

In [ ]:
### merged into 16 cell types
df_merged<-as.data.frame(dec_mtx_conf.t_nor)
df_merged$Vascular<-rowSums(df_merged[,c("A.Endothelial","C.Endothelial",'Pericyte','Tip.Cell','U.Vascular','V.Endothelial','VSMC')])
df_merged$OPC<-rowSums(df_merged[,c('Div.OPC','OPC.A','OPC.B','Pre.Oligo')])
df_merged$Macrophage<-rowSums(df_merged[,c('Border.Associated.Mac','Chemotaxis.Inducing.Mac','Inflammatory.Mac')])
#H_Microglia<-apply(df_merged[,c('Homeostatic.Microglia')],1,sum)
df_merged$R.Microglia<-rowSums(df_merged[,c('Dividing.Microglia','Inflammatory.Microglia','Migrating.Microglia')])
df_merged$Myeloid<-rowSums(df_merged[,c('Dividing.Myeloid','Interferon.Myeloid')])
df_merged$Ependymal<-rowSums(df_merged[,c('Astroependymal','Ependymal.A','Ependymal.B')])
df_merged<-df_merged[,-which(colnames(df_merged)%in%c("A.Endothelial","C.Endothelial",'Pericyte','Tip.Cell','U.Vascular','V.Endothelial','VSMC',
                                                        'Div.OPC','OPC.A','OPC.B','Pre.Oligo',
                                                        'Border.Associated.Mac','Chemotaxis.Inducing.Mac','Inflammatory.Mac',
                                                        #'Homeostatic.Microglia',
                                                        'Dividing.Microglia','Inflammatory.Microglia','Migrating.Microglia',
                                                        'Dividing.Myeloid','Interferon.Myeloid',
                                                        'Astroependymal','Ependymal.A','Ependymal.B'))]
colnames(df_merged)
colSums(df_merged)

In [45]:
write.csv(df_merged,"WT.fc1.fcreg1_subweights.decon_conf.nor.merged16celltypes.csv")

In [ ]:
meta<-read.csv("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.SCT.pc20.k50.res02.meta.data.csv")
rownames(meta)<-meta[,1]
head(meta)

In [ ]:
meta_dec<-cbind(meta,df_merged)
colnames(meta_dec)

In [49]:
write.csv(meta_dec,"16celltypes.dec_conf.nor.meta.csv")

In [ ]:
meta_dec_all<-cbind(meta,dec_mtx_conf.t_nor)
colnames(meta_dec_all)

In [55]:
write.csv(meta_dec_all,"32celltypes.dec_conf.nor.meta.csv")

In [41]:
all_celltypes<-c('A.Endothelial','Astro.Svep1','Astrocyte.Gfap','Astrocyte.Slc7a10','Astroependymal',
                 'Border.Associated.Mac','C.Endothelial','Chemotaxis.Inducing.Mac','Dendritic','Div.OPC',
                 'Dividing.Microglia','Dividing.Myeloid','Ependymal.A','Ependymal.B','Fibroblast',
                 'Homeostatic.Microglia','Inflammatory.Mac','Inflammatory.Microglia','Interferon.Myeloid',
                 'Migrating.Microglia','Monocyte','Neuron','Neutrophil','Oligodendrocyte',
                 'OPC.A','OPC.B','Pericyte','Pre.Oligo','Tip.Cell','U.Vascular','V.Endothelial','VSMC')
celltypes<-c('Neuron','Oligodendrocyte','OPC','Ependymal','Vascular','Fibroblast','Macrophage','Myeloid',
                'Monocyte','Dendritic','Neutrophil','Homeostatic.Microglia','R.Microglia','Astrocyte.Gfap','Astrocyte.Slc7a10',
    'Astro.Svep1')
#celltypes2<-c('Neuron','Oligodendrocyte','OPC','Ependymal','Vascular','Fibroblast','Macrophage','Myeloid',
#                'Monocyte','Homeostatic.Microglia','R.Microglia','Astrocyte.Gfap','Astrocyte.Slc7a10')

In [56]:
sf<-"32celltypes.conf_spatial/"
if(!dir.exists(sf))
    dir.create(sf)

options(repr.plot.width=20,repr.plot.height=16)
for(i in all_celltypes){
    ggplot(meta_dec_all,aes_string("x.ad","y.ad",color=i))+
    geom_point(size=1)+
    scale_color_gradientn(colours = colorRampPalette(rev(brewer.pal(n=11,name="Spectral")))(50))+
    xlab(paste0(""))+
    ylab(paste0(""))+
    theme(panel.background = element_blank(),
         panel.grid.major=element_blank(),
         panel.grid.minor=element_blank(),
          axis.text.x=element_blank(),
          axis.text.y=element_blank()
         )
    ggsave(paste0(sf,i,".spatialplot.png"),width=20,height=16,dpi=300)
}

In [53]:
sf<-"16celltypes.conf_spatial/"
if(!dir.exists(sf))
    dir.create(sf)

options(repr.plot.width=20,repr.plot.height=16)
for(i in celltypes){
    ggplot(meta_dec,aes_string("x.ad","y.ad",color=i))+
    geom_point(size=1)+
    scale_color_gradientn(colours = colorRampPalette(rev(brewer.pal(n=11,name="Spectral")))(50))+
    xlab(paste0(""))+
    ylab(paste0(""))+
    theme(panel.background = element_blank(),
         panel.grid.major=element_blank(),
         panel.grid.minor=element_blank(),
          axis.text.x=element_blank(),
          axis.text.y=element_blank()
         )
    ggsave(paste0(sf,i,".spatialplot.png"),width=20,height=16,dpi=300)
}

### stacked area plot

In [ ]:
col1<-alpha(c('#000075','#008080','#ffe119','#800000',#'#bcf60c',,
        #
        #'
        '#aaffc3',
        #'#808080',
                            '#46f0f0' ,
        #
        '#e6194b',
        #
        '#f58231',
        
        #'#9a6324',#'#fffac8',
        '#808000',
        #'#3cb44b',
        
        '#fabebe',
        
        
        '#fffac8','#ffd8b1',
              '#4363d8','#911eb4','#e6beff','#f032e6'
      ),0.8)
show_col(col1)

#col2<-alpha(c('#000075','#008080','#ffe119','#800000',#'#bcf60c',#'#fffac8',#'#ffd8b1',#'#4363d8',
#        #
#        #'
#        '#aaffc3',
#        #'#808080',
#                            '#46f0f0' ,
#        #
#        '#e6194b',
#        #
#        '#f58231',
#        '#e6beff',
#        #'#9a6324',#'#fffac8',
#        '#808000',
#        #'#3cb44b',
#        
#        '#fabebe',
#        
#        '#911eb4',
#        '#f032e6'
#      ),0.8)
#col2
#show_col(col2)

In [499]:
mycol<-col1
names(mycol)<-celltypes

In [ ]:
df<-meta_dec[,c("time",celltypes)]
head(df)
df2<-df %>% gather(key = "cell_type",value = "total_proportion",-c(1))
head(df2)
df3<-df2 %>% group_by(time,cell_type) %>% summarise(n=sum(total_proportion)) %>% mutate(percentage=n/sum(n))
head(df3)
df3$time2<-NA
df3[grepl("WT_sham",df3$time,fixed = T),"time2"]<-1
df3[grepl("WT_3h",df3$time,fixed = T),"time2"]<-2
df3[grepl("WT_24h",df3$time,fixed = T),"time2"]<-3
df3[grepl("WT_72h",df3$time,fixed = T),"time2"]<-4
head(df3)
df3$cell_type<-factor(df3$cell_type,levels=c(celltypes))
options(repr.plot.width=8,repr.plot.height=6)
ggplot(df3,aes(x=time2,y=percentage,fill=cell_type))+
    geom_area(alpha=1,size=0.2,color="black")+
    #scale_fill_viridis(discrete = T,option = "D")
    #scale_color_npg()
    #scale_color_discrete()
    scale_fill_manual(values =col1)+
    theme_bw()+
    theme(panel.grid.major = element_blank(),panel.grid.minor = element_blank(),axis.title = element_blank()
         )
ggsave("WT.fc1.fcreg1.conf5.subweights.16celltypes.time.stacked.area.plot_220325.png",width=8,height = 6,dpi=300)

### dotplot

In [131]:
da<-readRDS("/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.rds")

In [ ]:
colnames(da@meta.data)
da@meta.data<-meta_dec
colnames(da@meta.data)

In [135]:
da$orig.ident<-factor(da$orig.ident,levels = c('WT_sham_H_R2_1mm',
                                               'WT_sham_H_R2',
                                               'WT_sham_T_210323',
                                               'WT_sham_T_R2_1mm',
                                               'WT_3h_H_R2_1mm',
                                               'WT_3h_H_R2',
                                               'WT_3h_T_R2',
                                               'WT_3h_T_210330_1mm',
                                               'WT_24h_H_R1_1mm',
                                               'WT_24h_H_R1',
                                               'WT_24h_T_201231',
                                               'WT_24h_T_R1_1mm',
                                               'WT_72h_H_R1_1mm',
                                               'WT_72h_H_210323',
                                               'WT_72h_T_R2',
                                               'WT_72h_T_R1_1mm'))

In [ ]:
p3 <- DotPlot(da, features = celltypes2, group.by = 'orig.ident', scale = F) + 
    scale_color_gradientn(colours = rev(brewer.pal(9,"Spectral")))+
      theme(axis.text.x = element_text(angle = 90, size = 12, margin = margin(10,0,0,0)),
            axis.title = element_blank())
options(repr.plot.width = 8, repr.plot.height = 8)
p3
ggsave("WT.RCTD.fc1.fcreg1.conf5.13CelltypesByident.Dotplot.png", width = 8, height = 8)
#pdf("WT_merge_replace_v2.SCT.reg6.RCTD.uninjRef.Dotplot.pdf", width = 15, height = 7)
#p3
#dev.off()

### SpatialPie chart

In [536]:
da<-readRDS("/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.rds")


In [541]:
da@meta.data<-meta_dec

In [543]:
sf<-"SpatialPie_16celltypes/"
if(!dir.exists(sf))
    dir.create(sf)
slice <- Images(da)
options(repr.plot.width = 15, repr.plot.height = 15)
for(i in 1:length(slice)){
    p1 <- SpatialScatterPie(da,
                  cell_types_all = celltypes,
                  images=slice[i],       
                  cols = mycol,
                  pie.scale = 0.4)
    p1
    ggsave(paste(sf,"WT.deconvByRCTD_spatialPie.",slice[i],".png", sep = ""), p1, 
           width = 15, height = 15,dpi=300)
}